In [ ]:
from google.colab import drive

drive.mount('/drive')

Mounted at /drive


In [ ]:
#!git clone https://github.com/XuandongZhao/Unigram-Watermark

In [ ]:
cd '/drive/My Drive/Colab Notebooks/ISO/'

/drive/.shortcut-targets-by-id/1k2JFo63MNzskIgAHpOe33d-BrKTiK7mE/ISO


In [ ]:
cd zhao-Unigram-Watermark

/drive/.shortcut-targets-by-id/1k2JFo63MNzskIgAHpOe33d-BrKTiK7mE/ISO/zhao-Unigram-Watermark


In [ ]:
!pip install -q -r new_requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 57.6 MB/s eta 0:00:00


In [ ]:
import json
import argparse
import json
import os
import pandas as pd
from tqdm import tqdm
import torch
from transformers import AutoTokenizer
from gptwm import GPTWatermarkDetector

In [ ]:
checkpoint = 'facebook/opt-1.3b'

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

In [ ]:
detector = GPTWatermarkDetector(fraction=0.5,
                                strength=2.0,
                                vocab_size=50272,
                                watermark_key=0)

In [ ]:
cd '../instances'

/drive/.shortcut-targets-by-id/1k2JFo63MNzskIgAHpOe33d-BrKTiK7mE/ISO/instances


In [ ]:
def calculate_scores(filename, keyword, custom_filename=None, threshold=6):
    detection_filename = (filename[:-5] + '_detection.json') if custom_filename == None else custom_filename
    df = pd.read_json(filename, lines=True)
    detected = 0
    not_detected = 0
    with open(detection_filename, "r+" if os.path.exists(detection_filename) else "w+") as f:
        with torch.no_grad():
            for i in tqdm(range(len(f.readlines()), len(df["input"]))):
                line = (df["input"][i] if keyword == "continuation" else "") + df[keyword][i]

                gen_tokens = tokenizer(line, add_special_tokens=False)["input_ids"]
                z_score = detector.detect(gen_tokens)
                wm_pred = 1 if z_score > threshold else 0

                _line = df["input"][i].replace('\n', '\\n').replace('"', '\\"')

                f.write(f'{{"input": "{_line}", "z_score": "{z_score}", "prediction": "{wm_pred}"}}\n')

                if wm_pred:
                    detected += 1
                else:
                    not_detected += 1

    print("\nDetected:", detected)
    print("Not detected:", not_detected)

In [ ]:
calculate_scores("zhao_no_attack.json", "continuation")

100%|██████████| 512/512 [00:05<00:00, 96.62it/s] 


Detected: 438
Not detected: 74


In [ ]:
calculate_scores("zhao_paraphrase.json", "paraphrase")

100%|██████████| 512/512 [00:03<00:00, 164.21it/s]


Detected: 43
Not detected: 469


In [ ]:
calculate_scores('zhao_backtranslated.json', "translation")

100%|██████████| 512/512 [00:04<00:00, 126.06it/s]


Detected: 142
Not detected: 370


In [ ]:
calculate_scores('c4_selection.json', "continuation", "zhao_c4_detection.json")

100%|██████████| 512/512 [00:03<00:00, 161.59it/s]


Detected: 0
Not detected: 512
